## Rules
In salary cap contests, participants will create a lineup by selecting players listed in the Player Pool. Each player listed has an assigned salary and a valid lineup must not exceed the salary cap of $50,000.

Contest results will be determined by the total points accumulated by each individual lineup entry (scoring rules summarized below).

Participation in each contest must be made only as specified in the Terms of Use. Failure to comply with these Terms of Use will result in disqualification and, if applicable, prize forfeiture.

## Goals
Classic Game Types
Full Team Contests

In [435]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import floor

In [1]:
import pandas as pd
from random import sample

In [2]:
drop_attribs = [
        "injury_status",
        "game_date",
        "opp",
        "spread",
        "over_under",
        "implied_team_score",
        "L5_dvp_rank",
        "L5_ppg_floor",
        "L5_ppg_avg",
        "L5_ppg_max",
        "Rest",
        "Opp Pace",
        "Opp DEff",
        "Opp DvP",
        "L2 FGA",
        "L5 FGA",
        "S FGA",
        "L2 Min",
        "L5 Min",
        "S Min",
        "L5 FP",
        "S FP",
        "Floor FP",
        "Ceil FP",
        'ppg_projection', 'value_projection', 'Proj FP', 'Proj Min', 'Proj Val',
        "position",
        "salary",  
]

In [3]:
def reduceForProjectionAnalysis(file):
    file['FantasyFuelPPGProj'] = file['ppg_projection']
    file['FantasyFuelValueProj'] = file['value_projection']
    file['DFN_PPGProj'] = file['Proj FP']
    file['DFN_MinProj'] = file['Proj Min']
    file['DFN_ValueProj'] = file['Proj Val']
    file.drop(drop_attribs, axis=1, inplace=True)
    file = file.loc[file['Actual FP'] > 0]
    return file



In [436]:
##Helper Functions
def verifyLineup(lineup):
    enough_represented_teams = len(set(lineup['team'].to_list()))>=2
    under_salary_cap = lineup['Salary'].sum() <= 50000
    all_unique_players = len(set(lineup['Player Name'].to_list())) == 8
    if enough_represented_teams and under_salary_cap and all_unique_players:
        return True
    else:
        return False
    
def createRandomPopulation(point_guards, shooting_guards, small_forwards, power_forwards, guards, forwards, centers, util, limit):
    counter = 0
    lineups = []
    while(counter<limit):
        lineup = pd.DataFrame()
        lineup = lineup.append(point_guards.sample(n=1))
        lineup = lineup.append(shooting_guards.sample(n=1))
        lineup = lineup.append(small_forwards.sample(n=1))
        lineup = lineup.append(power_forwards.sample(n=1))
        lineup = lineup.append(guards.sample(n=1))
        lineup = lineup.append(forwards.sample(n=1))
        lineup = lineup.append(centers.sample(n=1))
        lineup = lineup.append(util.sample(n=1))
        if(verifyLineup(lineup)):
            lineups.append(lineup)
            counter = counter + 1
    return lineups


def mate(lineup1, lineup2):
    point_guards = lineup1.loc[lineup1['Pos'].str.contains("PG")].append(lineup2.loc[lineup2['Pos'].str.contains("PG")])
    shooting_guards = lineup1.loc[lineup1['Pos'].str.contains("SG")].append(lineup2.loc[lineup2['Pos'].str.contains("SG")])
    small_forwards = lineup1.loc[lineup1['Pos'].str.contains("SF")].append(lineup2.loc[lineup2['Pos'].str.contains("SF")])
    power_forwards = lineup1.loc[lineup1['Pos'].str.contains("PF")].append(lineup2.loc[lineup2['Pos'].str.contains("PF")])
    guards = lineup1.loc[lineup1['Pos'].str.contains("G")].append(lineup2.loc[lineup2['Pos'].str.contains("G")])
    forwards = lineup1.loc[lineup1['Pos'].str.contains("F")].append(lineup2.loc[lineup2['Pos'].str.contains("F")])
    centers = lineup1.loc[lineup1['Pos'].str.contains("C")].append(lineup2.loc[lineup2['Pos'].str.contains("C")])
    util = lineup1.append(lineup2)
    child_lineup = lineup1.append(lineup2)
    while(not verifyLineup(child_lineup)):
        child_lineup = pd.DataFrame()
        child_lineup = child_lineup.append(point_guards.sample(n=1))
        child_lineup = child_lineup.append(shooting_guards.sample(n=1))
        child_lineup = child_lineup.append(small_forwards.sample(n=1))
        child_lineup = child_lineup.append(power_forwards.sample(n=1))
        child_lineup = child_lineup.append(guards.sample(n=1))
        child_lineup = child_lineup.append(forwards.sample(n=1))
        child_lineup = child_lineup.append(centers.sample(n=1))
        child_lineup = child_lineup.append(util.sample(n=1))
    return child_lineup


def sortFitest(population): 
    population.sort(key=actualPointSum, reverse=True)
    population = eliminateDuplicates(population)
    return population 



def eliminateDuplicates(population):
    unique_lineups = set()
    unique_population = []
    for lineup in population:
        lineup_set = set()
        for player in lineup['Player Name'].tolist():
            lineup_set.add(player)
            
        if(not lineup_set.issubset(unique_lineups)):  
            unique_population.append(lineup)
            unique_lineups = unique_lineups.union(lineup_set)
    return unique_population


        

def actualPointSum(lineup):
    return lineup['Actual FP'].sum() 


In [498]:
def evaluateFitness(population):
    sum = 0 
    for lineup in population[:20]:
        sum = sum + lineup['Actual FP'].sum()
    return sum/20


def performSelection(population):
    selected_population = population[:1]
    selected_population.extend(sample(population[1:10], 5))
    #selected_population.extend(sample(population[10:20], 4))
    selected_population.extend(sample(population, 9))
    #selected_population.extend(sample(population[50:], 10))
    return selected_population


def performCrossover(population):
    counter = 0
    children = []
    while(counter < 10):
        parents = sample(population,2)
        children.append(mate(parents[0],parents[1]))
        counter = counter+1
    return children
        

def createNextGeneration(old_population):
    selected_population = performSelection(old_population)
    children = performCrossover(selected_population)
    new_generation = old_population[:len(old_population)-10]
    new_generation.extend(children)
    return new_generation
    
        

In [499]:
def geneticAlgorithmForHistorialBestLineups2(file):
    population_size = 50
    point_guards = file.loc[file['Pos'].str.contains("PG")]
    shooting_guards = file.loc[file['Pos'].str.contains("SG")]
    small_forwards = file.loc[file['Pos'].str.contains("SF")]
    power_forwards = file.loc[file['Pos'].str.contains("PF")]
    guards = file.loc[file['Pos'].str.contains("G")]
    forwards = file.loc[file['Pos'].str.contains("F")]
    centers = file.loc[file['Pos'].str.contains("C")]
    util = file
    population = createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, population_size)  
    counter = 0
    while(counter<100):
        currentPopulation = sortFitest(population)
        population = createNextGeneration(currentPopulation)
        print(len(population))
        population.extend(createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, 50 - len(population)))
        print(population[0]["Actual FP"].sum())
        counter = counter+1
    print("*****************")
    return currentPopulation

In [502]:
dates = [
    
]


def saveBestLineups_toCSV(file, date):
    file_name = 'BestLineups/{}'.format(date)
    folded_file = pd.concat(file).reset_index()
    folded_file.to_csv(file_name)

def createBestLineups():
    for game_day in dates:
        file_name = 'HistoricalData_Merged/{}'.format(game_day[1])
        cleaned_file = reduceForProjectionAnalysis(pd.read_csv(file_name))
        best_population = geneticAlgorithmForHistorialBestLineups2(cleaned_file)
        saveBestLineups_toCSV(best_population, game_day[1])
        
createBestLineups()

##main method 
#population_size = 100

def geneticAlgorithmForHistorialBestLineups(file):
    point_guards = file.loc[file['Pos'].str.contains("PG")]
    shooting_guards = file.loc[file['Pos'].str.contains("SG")]
    small_forwards = file.loc[file['Pos'].str.contains("SF")]
    power_forwards = file.loc[file['Pos'].str.contains("PF")]
    guards = file.loc[file['Pos'].str.contains("G")]
    forwards = file.loc[file['Pos'].str.contains("F")]
    centers = file.loc[file['Pos'].str.contains("C")]
    util = file
    
    population = createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, population_size)
    prevGenFitness = -100
    currentGenFitness = 0
    
    while(currentGenFitness-prevGenFitness>1):
        currentPopulation = sortFitest(population)
        prevGenFitness = currentGenFitness
        currentGenFitness = evaluateFitness(currentPopulation)
        population = createNextGeneration(currentPopulation)
        population.extend(createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, 10))
        print(population[0]["Actual FP"].sum())
    return currentPopulation

test = reduceForProjectionAnalysis(pd.read_csv('HistoricalData_Merged/2020-01-24'))
pop = geneticAlgorithmForHistorialBestLineups(test)



def eliminateDuplicates(population):
    unique_lineups = set()
    unique_population = []
    for lineup in population:
        if lineup['Actual FP'].sum() not in unique_lineups:
            unique_lineups.add(lineup['Actual FP'].sum())
            unique_population.append(lineup)
    return unique_population
    
def printDistib(population):
    h = []
    for x in population:
        print(x['Actual FP'].sum())
        h.append(x['Actual FP'].sum())

    std = np.std(h) 
    mean = np.mean(h)
    print(mean)
    print(std)
    print(mean-std)
    plt.plot(norm.pdf(h,mean,std))
    
    
test['Player Name']

se = set()
for lineup in pop2:
    for player in lineup['Player Name'].tolist():
        se.add(player)
    
len(list(se))